In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import cv2
import json
import glob
import torch
import tifffile
import skimage.io
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

sys.path.append("../code/")

In [ ]:
from params import *
from utils.rle import *

from data.dataset import load_image

from utils.metrics import dice_scores_img
from utils.plots import plot_heatmap_preds, plot_contours_preds

### Load

In [ ]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train_2_onlyfc.csv")
# df_mask_old = pd.read_csv(DATA_PATH + "train_4.csv")
df_mask_new = pd.read_csv(DATA_PATH + "train_2_fix.csv")
# df = pd.read_csv(OUT_PATH + "df_images.csv")

### Data

In [ ]:
root = TIFF_PATH_2
rle_path = DATA_PATH + "train_2.csv"
reduce_factor = 1
rles = pd.read_csv(rle_path)

### Experiment

In [ ]:
log_folder = "../logs/2021-05-06/1/"  # b1

In [ ]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [ ]:
global_threshold = 0.5

In [ ]:
preds = glob.glob(log_folder + "pred_*.npy")

## Train viz

In [ ]:
THRESHOLD = 0.5

In [ ]:
NAMES = [
    "b9a3865fc",
    "aaa6a05cc",
    "e79de561c",
    "8242609fa",
    "2f6ecfcdf",
    "0486052bb",
    "26dc41664",
    "afa5e8098",
    "54f2eec69",
    "cb2d976f4",
    "4ef6695ce",
    "095bf7a1f",
    "1e2425f28",
    "c68fe75ea",
    "b2dc8411c",
]

In [ ]:
# for mask_name in df_mask['id']:
#     img = load_image(os.path.join(TIFF_PATH_2, mask_name + ".tiff"), full_size=False, reduce_factor=2)
#     rle = df_mask[df_mask['id'] == mask_name]['encoding']
#     mask = enc2mask(rle, (img.shape[1], img.shape[0]))
#     rle_new = df_mask_new[df_mask_new['id'] == mask_name]['encoding']
#     mask_new = enc2mask(rle_new, (img.shape[1], img.shape[0]))
#     score = dice_scores_img(mask_new , mask)
#     print(f'Score for downscaled image {mask_name} is {score:.4f}')

In [ ]:
# 2f6ecfcdf , b2dc8411c,
mask_name = "b2dc8411c" 

idx = [i for i, path in enumerate(preds) if mask_name in path][0]

In [ ]:
probs = np.load(preds[idx]).astype(np.float32)

if len(probs.shape) == 3:
    probs = probs.mean(0)

pred = (probs > THRESHOLD).astype(np.uint8)

In [ ]:
img = load_image(os.path.join(TIFF_PATH_2, mask_name + ".tiff"), full_size=False, reduce_factor=2)

In [ ]:
df_mask = pd.read_csv(DATA_PATH + "train_2.csv")
df_mask_new = pd.read_csv(DATA_PATH + "train_2_fix.csv")

In [ ]:
rle = df_mask[df_mask['id'] == mask_name]['encoding']
mask = enc2mask(rle, (img.shape[1], img.shape[0]))

In [ ]:
rle_new = df_mask_new[df_mask_new['id'] == mask_name]['encoding']
mask_new = enc2mask(rle_new, (img.shape[1], img.shape[0]))

In [ ]:
# rle_old = df_mask_old[df_mask_old['id'] == mask_name]['encoding']
# mask_old = enc2mask(rle_old, (img.shape[1], img.shape[0]))

In [ ]:
# score = dice_scores_img(pred , mask)
# print(f'Score for downscaled image is {score:.4f}')

In [ ]:
# score = dice_scores_img(mask_new , mask)
# print(f'Score for downscaled image is {score:.4f}')

### Plot

In [ ]:
w = 1000
h = int(w *  mask.shape[0] / mask.shape[1])

In [ ]:
skimage.io.imsave(log_folder + f"viz_{mask_name}.png", (img * 255).astype(np.uint8))

In [ ]:
fig, img = plot_contours_preds(img, pred, mask, w=2, downsize=2)

skimage.io.imsave(log_folder + f"viz_{mask_name}.png", (img * 255).astype(np.uint8))

fig.update_layout(
    autosize=False,
    width=w,
    height=h,
)

fig.show()

In [ ]:
# fig = plot_heatmap_preds(img, probs, mask, w=1, downsize=2)

# fig.update_layout(
#     autosize=False,
#     width=w,
#     height=h,
# )

# fig.show()

## Test viz

### Load image

In [ ]:
df_test = pd.read_csv(DATA_PATH + "sample_submission.csv")

list(df_test['id'])

In [ ]:
name = 'aa05346ff'
fold = 0

img = load_image(f'{DATA_PATH}/test_4/{name}.tiff', full_size=False)

In [ ]:
THRESHOLD = 0.5

In [ ]:
probs = np.load(log_folder + f'pred_{name}_{fold}.npy').astype(np.float32)
pred = (probs > THRESHOLD).astype(np.uint8)

## From sub

In [ ]:
sub = pd.read_csv(OUT_PATH + "submission_combine.csv")
sub2 = pd.read_csv(OUT_PATH + "submission_tta.csv")

for id_ in list(df_test['id']):
    shape = df_info[df_info['image_file'] == id_ + '.tiff'][['width_pixels', 'height_pixels']].values[0]
    
    rle = sub[sub['id'] == id_]['predicted'].values
    sub_mask = enc2mask(rle, shape)
    
    rle_2 = sub2[sub2['id'] == id_]['predicted'].values
    sub_mask_2 = enc2mask(rle_2, shape)
    
    print(f'{id_} :  {dice_scores_img(sub_mask, sub_mask_2) :.3f}')

In [ ]:
shape = df_info[df_info['image_file'] == name + '.tiff'][['width_pixels', 'height_pixels']].values[0]

In [ ]:
sub = pd.read_csv(OUT_PATH + "submission_combine.csv")
rle = sub[sub['id'] == name]['predicted'].values

sub_mask = enc2mask(rle, shape)

sub_mask = cv2.resize(
    sub_mask,
    tuple(list(shape // 4)),
    interpolation=cv2.INTER_NEAREST,
)

In [ ]:
sub = pd.read_csv(OUT_PATH + "submission_tta.csv")
rle = sub[sub['id'] == name]['predicted'].values

sub_mask2 = enc2mask(rle, shape)

sub_mask2 = cv2.resize(
    sub_mask2,
    tuple(list(shape // 4)),
    interpolation=cv2.INTER_NEAREST,
)

### Plot

In [ ]:
w = 1000
h = int(w * pred.shape[0] / pred.shape[1])

In [ ]:
fig, _ = plot_contours_preds(img, sub_mask, mask=sub_mask2, w=2, downsize=2)
# fig = plot_contours_preds(img, sub_mask, mask=sub_mask2, w=2, downsize=2)

fig.update_layout(
    autosize=False,
    width=w,
    height=h,
)

fig.show()

## Extra data

In [ ]:
df_extra = pd.read_csv('../input/train_extra.csv').set_index('id')
DATA_PATH_EXTRA = DATA_PATH + "extra_tiff/"

In [ ]:
name = "SAS_21908_001"

In [ ]:
rle = df_extra.loc[name]

In [ ]:
img = tifffile.imread(DATA_PATH_EXTRA + name + ".tiff").squeeze()

In [ ]:
mask = enc2mask(rle, img.shape[:2][::-1])

In [ ]:
w = 1000
h = int(w * mask.shape[0] / mask.shape[1])

fig = plot_contours_preds(img, mask, w=2, downsize=4)

fig.update_layout(
    autosize=False,
    width=w,
    height=h,
)

fig.show()